# Experiments

- Datasets:
    - ERA-Interim   (39 years)
    - ERA5          (40 years)
    - EC-Earth      (?? years)
- Train Test:
    - Leave 4 year out Cross Validation
- Methods:
    - McKinnon (PEP)
    - Spatial Covariance (SST)
    - Gradient Boosting Regressor (SST + Indices)

# EC-Earth
Om een goede spreiding van El-Niño/La Niña te krijgen, willen we alle 16 initialisaties van de oceaan gebruiken. Om interjaarlijste variabliteit en model-spin-up problemen te vermijden, kiezen we van elke run enkel het laatste jaar van de 5. Dit om de meest onafhankelijk mogelijke jaren te kiezen.

In [1]:
import sys; sys.path.append('../..')
from heatwave import ERA, Country, utils

import matplotlib.pyplot as plt

from netCDF4 import Dataset, num2date
import numpy as np

import os

In [36]:
INPUT = '/Volumes/Samsung_T5/Thesis/EC-Earth'
OUTPUT = os.path.join(INPUT, '160')

STARTS = range(1, 17)
MEMBERS = range(0, 10)
YEARS = [2039]

# # TOS
# VAR = 'tos'
# FORMAT = 'tos_PD/tos_d_s{0:02d}/tos_d_ECEarth_PD_s{0:02d}r{1:02d}_{2:04d}.nc'
# INDEX = [None, slice(8, 90)]

# # TAS
# VAR = 'tas'
# FORMAT = 'tas_PD_US/tas_d_ECEarth_PD_s{0:02d}r{1:02d}_{2:04d}_US.nc'
# INDEX = []

# # MRLSL
# VAR = 'mrlsl4'
# FORMAT = VAR+'_PD/'+VAR+'_d_s{0:02d}_US/'+VAR+'_d_ECEarth_PD_s{0:02d}r{1:02d}_{2:04d}_US.nc'
# INDEX = []

# PSL
VAR = 'psl'
FORMAT = 'psl_PD/psl_d_s{0:02d}/psl_d_ECEarth_PD_s{0:02d}r{1:02d}_{2:04d}.nc'
INDEX = [None, slice(8, 90)]

OUTPUT = os.path.join(OUTPUT, VAR+'_d_ECEarth_PD_1start_10ens_lastyr.nc')

KWARGS = dict(longitude_key='lon', latitude_key='lat', time_unit='D', time_origin='2000-01-01')

In [37]:
def walk(root: str, path_format: str, starts: list, members: list, years: list):
    for start in starts:
        for member in members:
            for year in years:
                yield os.path.join(root, path_format.format(start, member, year))

In [38]:
if os.path.exists(OUTPUT): os.remove(OUTPUT)

dataset = Dataset(OUTPUT, 'w', format='NETCDF4_CLASSIC')

time_index = 0

for path in walk(INPUT, FORMAT, STARTS, MEMBERS, YEARS):
    
    print(f"\rProcessing {path}", end="")
    
    data = ERA(path, VAR, index=INDEX, **KWARGS)
    
    if time_index == 0:
        time = dataset.createDimension('time', None)
        lat = dataset.createDimension('lat', data.shape[1])
        lon = dataset.createDimension('lon', data.shape[2])
        
        times = dataset.createVariable('time', np.float32, ('time',))
        latitudes = dataset.createVariable('lat', np.float32, ('lat',))
        longitudes = dataset.createVariable('lon', np.float32, ('lon',))
        var = dataset.createVariable(VAR, np.float32, ('time','lat','lon'))
        
        latitudes[:] = data.latitude
        longitudes[:] = data.longitude
        
        # Metadata
        var.author = data.dataset.author
        var.experiment = data.dataset.experiment
        
        times.units = "days since 2000-01-01 12:00:00"
        times.calendar = 'noleap'
        
        latitudes.units = data.dataset['lat'].units
        longitudes.units = data.dataset['lon'].units
        var.units = data.dataset[VAR].units
        
    time_values = data.time.values.ravel() + time_index
    time_length = len(time_values)
        
    var[time_index:time_index+time_length, :, :] = data.data
    times[time_index:time_index+time_length] = time_values
    
    time_index += time_length

dataset.close()

Processing /Volumes/Samsung_T5/Thesis/EC-Earth/psl_PD/psl_d_s16/psl_d_ECEarth_PD_s16r09_2039.nc

In [39]:
data = ERA(OUTPUT, VAR, **KWARGS)

data.latitude

# utils.plot_earth()
# plt.pcolormesh(data.longitude, data.latitude, data.data[0])
# plt.show()

array([ 80.18531   ,  79.06398   ,  77.94263   ,  76.82124   ,
        75.699844  ,  74.57843   ,  73.45701   ,  72.33558   ,
        71.214134  ,  70.09269   ,  68.97124   ,  67.849785  ,
        66.728325  ,  65.606865  ,  64.4854    ,  63.363934  ,
        62.242462  ,  61.12099   ,  59.99952   ,  58.878044  ,
        57.75657   ,  56.635094  ,  55.513615  ,  54.392136  ,
        53.270657  ,  52.149174  ,  51.027695  ,  49.90621   ,
        48.78473   ,  47.663246  ,  46.541763  ,  45.42028   ,
        44.298794  ,  43.17731   ,  42.055824  ,  40.934338  ,
        39.81285   ,  38.691364  ,  37.56988   ,  36.44839   ,
        35.326904  ,  34.205418  ,  33.08393   ,  31.962444  ,
        30.840956  ,  29.719467  ,  28.597979  ,  27.47649   ,
        26.355003  ,  25.233515  ,  24.112024  ,  22.990536  ,
        21.869047  ,  20.747559  ,  19.62607   ,  18.50458   ,
        17.383091  ,  16.2616    ,  15.140112  ,  14.018622  ,
        12.897133  ,  11.775643  ,  10.654153  ,   9.53